In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn import linear_model
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

## Fuel Consupmtion Dataset - Regression

In [2]:
fuel = pd.read_csv('/Users/nmims/Desktop/Semester V/Machine Learning/Exercise 5/FuelConsumption.csv')
fuel.head()

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


### Data Preprocessing

In [3]:
print(fuel.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MODELYEAR                 1067 non-null   int64  
 1   MAKE                      1067 non-null   object 
 2   MODEL                     1067 non-null   object 
 3   VEHICLECLASS              1067 non-null   object 
 4   ENGINESIZE                1067 non-null   float64
 5   CYLINDERS                 1067 non-null   int64  
 6   TRANSMISSION              1067 non-null   object 
 7   FUELTYPE                  1067 non-null   object 
 8   FUELCONSUMPTION_CITY      1067 non-null   float64
 9   FUELCONSUMPTION_HWY       1067 non-null   float64
 10  FUELCONSUMPTION_COMB      1067 non-null   float64
 11  FUELCONSUMPTION_COMB_MPG  1067 non-null   int64  
 12  CO2EMISSIONS              1067 non-null   int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 108.5+ KB
None

In [4]:
#fuel['CYLINDERS'] = fuel['CYLINDERS'].astype('object')
print(fuel.MODELYEAR.unique())
fuel.drop('MODELYEAR', axis = 1, inplace = True)
fuel.drop('MODEL', axis = 1, inplace = True)
fuel.drop('MAKE', axis = 1, inplace = True)
fuel.head()

[2014]


,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [5]:
fuel = pd.get_dummies(fuel)
fuel.head()

# OR

# for i in fuel.describe(include = np.object).columns:
#     fuel[i] = fuel[i].astype('category')
#     fuel[i] = fuel[i].cat.codes
#     fuel[i] = fuel[i].astype('category')

,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,VEHICLECLASS_COMPACT,VEHICLECLASS_FULL-SIZE,VEHICLECLASS_MID-SIZE,...,TRANSMISSION_AV6,TRANSMISSION_AV7,TRANSMISSION_AV8,TRANSMISSION_M5,TRANSMISSION_M6,TRANSMISSION_M7,FUELTYPE_D,FUELTYPE_E,FUELTYPE_X,FUELTYPE_Z
0,2.0,4,9.9,6.7,8.5,33,196,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2.4,4,11.2,7.7,9.6,29,221,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2,1.5,4,6.0,5.8,5.9,48,136,1,0,0,...,0,1,0,0,0,0,0,0,0,1
3,3.5,6,12.7,9.1,11.1,25,255,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3.5,6,12.1,8.7,10.6,27,244,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
fuel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 49 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ENGINESIZE                             1067 non-null   float64
 1   CYLINDERS                              1067 non-null   int64  
 2   FUELCONSUMPTION_CITY                   1067 non-null   float64
 3   FUELCONSUMPTION_HWY                    1067 non-null   float64
 4   FUELCONSUMPTION_COMB                   1067 non-null   float64
 5   FUELCONSUMPTION_COMB_MPG               1067 non-null   int64  
 6   CO2EMISSIONS                           1067 non-null   int64  
 7   VEHICLECLASS_COMPACT                   1067 non-null   uint8  
 8   VEHICLECLASS_FULL-SIZE                 1067 non-null   uint8  
 9   VEHICLECLASS_MID-SIZE                  1067 non-null   uint8  
 10  VEHICLECLASS_MINICOMPACT               1067 non-null   uint8  
 11  VEHI

In [7]:
fuel.head()

,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,VEHICLECLASS_COMPACT,VEHICLECLASS_FULL-SIZE,VEHICLECLASS_MID-SIZE,...,TRANSMISSION_AV6,TRANSMISSION_AV7,TRANSMISSION_AV8,TRANSMISSION_M5,TRANSMISSION_M6,TRANSMISSION_M7,FUELTYPE_D,FUELTYPE_E,FUELTYPE_X,FUELTYPE_Z
0,2.0,4,9.9,6.7,8.5,33,196,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2.4,4,11.2,7.7,9.6,29,221,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2,1.5,4,6.0,5.8,5.9,48,136,1,0,0,...,0,1,0,0,0,0,0,0,0,1
3,3.5,6,12.7,9.1,11.1,25,255,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3.5,6,12.1,8.7,10.6,27,244,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Regression

In [8]:
x = fuel.drop('CO2EMISSIONS', axis = 1)
y = fuel['CO2EMISSIONS']

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.6)

### Defining Models

In [9]:
linear = linear_model.LinearRegression()
ridge = linear_model.Ridge()
lasso= linear_model.Lasso()
elastic = linear_model.ElasticNet()
lasso_lars = linear_model.LassoLars()
bayes_ridge = linear_model.BayesianRidge()
logistics = linear_model.LogisticRegression()
sgd = linear_model.SGDClassifier()
passagg = linear_model.PassiveAggressiveClassifier()
ridgecv = linear_model.RidgeClassifierCV()
ridgeclass = linear_model.RidgeClassifier()

models_churn = [linear, ridge, lasso, elastic, lasso_lars, bayes_ridge, logistics, sgd, passagg, ridgecv, ridgeclass]

In [10]:
def get_cv_scores(model):
    scores = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))
    print('\n')

In [11]:
for i in models_churn:
    print(i)
    get_cv_scores(i)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
CV Mean:  -6.0941320751917285
STD:  0.616365732940854


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
CV Mean:  -6.201416701215591
STD:  0.7038892731907321


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
CV Mean:  -9.366467267428044
STD:  1.2080176531913387


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
CV Mean:  -20.79986987551532
STD:  2.1837674498451642


LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, max_iter=500, normalize=True, posi

In [12]:
alpha = [0.5, 1, 1.5, 0.01, 2.5, 0.0001, 10, 100, 0.35]
#solver = ['auto', 'svd']

param_grid = dict(alpha = alpha)#, solver = solver)

In [13]:
grids = GridSearchCV(estimator = lasso_lars, param_grid = param_grid, scoring = 'r2', cv = 10)
grid_result = grids.fit(x_train, y_train)

In [14]:
rnds = RandomizedSearchCV(estimator = ridge, param_distributions = param_grid, scoring='r2', cv = 10)
rnds_result = rnds.fit(x_train, y_train)

In [15]:
print(grid_result.best_params_)
print(rnds_result.best_params_)

{'alpha': 0.0001}
{'alpha': 0.01}


### Best Scores From Our Models

In [16]:
print(grid_result.best_score_)
print(rnds_result.best_score_)

0.9909428702658175
0.9909236153581424


## Iris Dataset - Classification

### Datasets Uploading and Preprocessing

In [17]:
from sklearn.datasets import load_iris
iris = load_iris()
data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
data1

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [18]:
x_iris = data1.drop('target', axis = 1)
y_iris = data1['target']
x1_train, x1_test, y1_train, y1_test = train_test_split(x_iris, y_iris, train_size = 0.7)
print(x1_train.shape)
print(x1_test.shape)
print(y1_train.shape)
print(y1_test.shape)

(105, 4)
(45, 4)
(105,)
(45,)


### Model Building

In [19]:
def get_cv_scores1(model):
    scores = cross_val_score(model, x1_train, y1_train, cv=5, scoring='accuracy')
    print('CV Mean Score: ', np.mean(scores))
    print('STD: ', np.std(scores))
    print('\n')

models_log = [logistics, sgd, passagg, ridgecv, ridgeclass]

In [20]:
for i in models_log:
    print(i)
    get_cv_scores1(i)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
CV Mean Score:  0.9428571428571428
STD:  0.04665694748158436


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
CV Mean Score:  0.8095238095238095
STD:  0.11664236870396087


PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_i

In [21]:
grid_list = []
random_list = []
for i in models_log:
    grids = GridSearchCV(estimator = lasso_lars, param_grid = param_grid, scoring = 'r2', cv = 10)
    grid_result = grids.fit(x_train, y_train)
    grid_list.append(grid_result.best_score_)
    rnds = RandomizedSearchCV(estimator = ridge, param_distributions = param_grid, scoring='r2', cv = 10)
    rnds_result = rnds.fit(x_train, y_train)
    random_list.append(rnds_result.best_score_)

### Best Scores From Our Models

In [22]:
print(max(grid_list))
print(max(random_list))

0.9909428702658175
0.9909236153581424
